In [39]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

list = os.listdir("Stocks1/")
number_files = len(list)

def archivo_no_vacio(fpath):
    return True if os.path.isfile(fpath) and os.path.getsize(fpath) > 0 else False

from collections import namedtuple


In [40]:
## hacemos un dict con los datos  de cada empresa

Empresa = namedtuple('Empresa', 'nombre industria sector marketcap')

empresas_df =  pd.read_csv("companies.csv", sep=",")
empresas_df["ticker"] = empresas_df["ticker"].str.lower()
# empresas_df

empresas_df = empresas_df[['ticker', 'company name', 'industry','sector', 'market cap']]
lista_nombres = empresas_df['company name'].tolist()
lista_tickers = empresas_df['ticker'].tolist()
lista_marketcap = empresas_df['market cap'].tolist()
lista_industrias = empresas_df['industry'].tolist()
lista_sectores = empresas_df['sector'].tolist()

dict_datos_empresas = dict()
for indice, ticker in enumerate(lista_tickers):

    nombre = lista_nombres[indice]
    industria = lista_industrias[indice]
    marketcap = lista_marketcap[indice]
    sector = lista_sectores[indice]

    dict_datos_empresas[ticker] = Empresa(nombre, industria, sector, marketcap)

# empresas_df


import json

with open("datos_empresas.json", "w") as file:
    dicto = json.dumps(dict_datos_empresas)
    file.write(dicto)

In [41]:
# con filtro fecha y filtro 500 empresas
list_files = os.listdir("Stocks1/")
dict_frames = dict() # dicccionario con todos los data frames que rellenaremos
dict_rendimientos = dict()

# no_sirven = 0
#recorremos todas las acciones en Stocks desde 2010 (Stocks1)
for nombre_archivo in list_files:

    # aplicamos filtro de 500 empresas
#     index = nombre_archivo.find(".us")
#     simple = nombre_archivo[0:index]
#     if simple not in list(empresas_df["Symbol"]):
#         continue

    path = f"Stocks1/{nombre_archivo}"

    # recorremos los archivos no vacios
    if archivo_no_vacio(path):

        df =  pd.read_csv(path, sep=",")
        df = df[['Date', 'Open', 'Close', 'Volume']]
        df['ticker'] = nombre_archivo.replace('.csv', '')
        df['Rendimiento'] = (df['Close'] - df['Open']) / df['Open']

        # quitamos los datos nulos
        df.dropna(subset=['Rendimiento'], inplace=True)

        df['Date']= pd.to_datetime(df['Date'], dayfirst = True)

        # filtramos para obtener los datos de hasta el 31 Dic de 2014
        df = df[(df['Date'] > '2015-01-01 00:00:00')]
        df = df[(df['Date'] < '2015-05-01 00:00:00')]

        # se guarda el nombre sin .txt
        dict_frames[nombre_archivo.split(".csv")[0]] = df
        dict_rendimientos[nombre_archivo.split(".csv")[0]] = df[['Rendimiento']]

In [43]:
# # encontremos la maxima presencia bursatil
# max_presencias = 0
# # hacemos un dict, key = nombre empresa, value = % presencia bursatil
# dict_presencias = dict()

# # calculo de la maxima presencia bursatil, valor que sera nuestra referencia
# #vemos las presencia de todas las acciones y elegimos la mayor
# for nombre, df in dict_frames.items():
#     # calculamos presencia bursatil de esa accion
#     pb = len(df.index)
#     if pb > max_presencias:
#         max_presencias = pb

# # guardamos el porcentaje de capitalizacion de cada empresa en el dict
# for nombre, df in dict_frames.items():
#     # calculamos presencia bursatil de esa accion
#     pb_porcentual = len(df.index)/max_presencias
#     dict_presencias[nombre] = pb_porcentual
## meter dicc presencias estandar archivo json ## 
with open('presencias.json') as json_file:
    dict_presencias = json.load(json_file)

In [44]:
# filtramos con aquellas empresas que cumplen con el 99,9% de las presencias
# creamos un nuevo dict para evitar cambios del dict en la iteracion
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
for accion, df in dict_frames.items():
    # si tiene menos del 99,9% de presencias se elimina
    if dict_presencias[accion] < 0.999:
        del dict_frames_nuevo[accion]


# actualizamos las acciones
dict_frames = dict_frames_nuevo

#imprimimos la cantidad de acciones


# agregamos los datos de las empresas
dict_frames_nuevo = dict_frames.copy()
no_sirven = 0
for accion, df in dict_frames.items():
    if accion.split('.us')[0] not in dict_datos_empresas.keys():
        dict_frames_nuevo.pop(accion)
        no_sirven += 1
        continue
    dict_frames_nuevo[accion]['Nombre'] = dict_datos_empresas[accion.split('.us')[0]].nombre
    dict_frames_nuevo[accion]['Sector'] = dict_datos_empresas[accion.split('.us')[0]].sector
#     df['Market Cap'] = dict_datos_empresas[accion.split('.us')[0]].marketcap
# dict_frames_nuevo['a.us']

# df_prueba = dict_frames_nuevo['a.us'].iloc[0]
# df_prueba2 = dict_frames_nuevo['a.us'].iloc[-1]
# df_prueba
# df_prueba2
industrias_utilizadas = []
dic_rango_open= {}
for item in dict_frames_nuevo:
    if item not in industrias_utilizadas:
        industrias_utilizadas.append(item)
        df_open_inicio = dict_frames_nuevo[item].iloc[0]
        df_open_final = dict_frames_nuevo[item].iloc[-1]
        dic_rango_open[item] = [df_open_inicio['Open'],df_open_final['Open'], df_open_final['Open']-df_open_inicio['Open'] ]
    else:
        continue
# dic_peso_industria= {'Industrials':1,'Real Estate':1,'Consumer Defensive':1,'Energy':1,'Basic Materials':1, 'Communication Services':1, ''}
suma_ganancias = 0
total_acciones = 0
for item in dic_rango_open:
    suma_ganancias = +float(dic_rango_open[item][2])
    total_acciones+=1
promedio = suma_ganancias/total_acciones
print(promedio)
## buscar si la key esta dentro del diccionario y si esta ponderarla por su peso 
### con pesos ## 
# suma_ganancias2 = 0
# total_acciones2 = 0

## tomar la diferencia de precios en dias extremosy multiplicarla por el peso usando la misma key para el dic de peso
#y dic de precios de open

-0.0030544747081712067


In [54]:
resultados_acciones ={"apu.us": 0.007499999999999983, "artna.us": 0.024999999999999967, "awk.us": 0.019938933922506133, "cdzi.us": 0.007499999999999983, "cpl.us": 0.02088758773231219, "duk.us": 0.020712910808316422, "ed.us": 0.018012446155815046, "kep.us": 0.007499999999999983, "ngg.us": 0.024999999999999967, "pam.us": 0.007499999999999983, "pcg.us": 0.007499999999999983, "ppl.us": 0.020643817591815435, "so.us": 0.024999999999999967, "sph.us": 0.016734093051900746, "tgs.us": 0.007499999999999983, "xel.us": 0.013070210737333639, "bti.us": 0.02500000000000004, "cl.us": 0.018719788426170014, "core.us": 0.019598221345834842, "dltr.us": 0.007500000000000005, "hsy.us": 0.02500000000000004, "iba.us": 0.012669773085624773, "kr.us": 0.02500000000000002, "lway.us": 0.007500000000000005, "mkc.us": 0.010173176419466572, "pg.us": 0.02500000000000002, "sam.us": 0.01633904072290378, "ths.us": 0.02500000000000002, "tues.us": 0.007500000000000005, "wmt.us": 0.02500000000000004, "bkt.us": 0.02500000000000004, "btz.us": 0.02500000000000004, "cb.us": 0.007500000000000005, "egf.us": 0.02500000000000004, "fdef.us": 0.007500000000000005, "jof.us": 0.009526078086907117, "mcr.us": 0.007500000000000005, "mfg.us": 0.007500000000000005, "mmc.us": 0.017251099815181373, "mqy.us": 0.007500000000000005, "mrln.us": 0.007500000000000005, "mui.us": 0.007500000000000005, "smfg.us": 0.008812630511370056, "swz.us": 0.021214966178268797, "tdf.us": 0.010483630680640321, "tree.us": 0.013121699758917036, "tst.us": 0.007500000000000005, "wiw.us": 0.02500000000000004, "wltw.us": 0.009589894968715217, "bt.us": 0.02500000000000004, "ccoi.us": 0.011299111248407023, "chl.us": 0.007500000000000005, "cht.us": 0.007500000000000005, "cmcsa.us": 0.02500000000000004, "dish.us": 0.019026661691407955, "idt.us": 0.007500000000000005, "lbtya.us": 0.022666865556218156, "lbtyk.us": 0.02500000000000004, "sbac.us": 0.009507361503966739, "tkc.us": 0.02500000000000004, "tu.us": 0.02500000000000004, "utsi.us": 0.007500000000000005, "vod.us": 0.02500000000000004, "vz.us": 0.007500000000000005, "parametros": [0.03, 0.1, 0.001, 0.05, 0.25, 0.001]}
print(type(dicdic))

<class 'dict'>


In [74]:

import json
with open("Resultados_acciones.json") as json_file:
    resultados_acciones = json.load(json_file)
resultados_acciones

{'apu.us': 0.007499999999999983,
 'artna.us': 0.024999999999999967,
 'awk.us': 0.019938933922506133,
 'cdzi.us': 0.007499999999999983,
 'cpl.us': 0.02088758773231219,
 'duk.us': 0.020712910808316422,
 'ed.us': 0.018012446155815046,
 'kep.us': 0.007499999999999983,
 'ngg.us': 0.024999999999999967,
 'pam.us': 0.007499999999999983,
 'pcg.us': 0.007499999999999983,
 'ppl.us': 0.020643817591815435,
 'so.us': 0.024999999999999967,
 'sph.us': 0.016734093051900746,
 'tgs.us': 0.007499999999999983,
 'xel.us': 0.013070210737333639,
 'bti.us': 0.02500000000000004,
 'cl.us': 0.018719788426170014,
 'core.us': 0.019598221345834842,
 'dltr.us': 0.007500000000000005,
 'hsy.us': 0.02500000000000004,
 'iba.us': 0.012669773085624773,
 'kr.us': 0.02500000000000002,
 'lway.us': 0.007500000000000005,
 'mkc.us': 0.010173176419466572,
 'pg.us': 0.02500000000000002,
 'sam.us': 0.01633904072290378,
 'ths.us': 0.02500000000000002,
 'tues.us': 0.007500000000000005,
 'wmt.us': 0.02500000000000004,
 'bkt.us': 0.025

In [66]:


sum_ganancias2 = 0
total_acciones2 = 0
for item in resultados_acciones:
    
    if item != "parametros":
        sum_ganancias2 = +float(dic_rango_open[item][2]*resultados_acciones[item])
        total_acciones2+=1

promedio = sum_ganancias2/total_acciones2

print(resultados_acciones["parametros"])
print(promedio)

0.03033000000000005
64
[0.03, 0.1, 0.001, 0.05, 0.25, 0.001]
0.0004739062500000008


In [38]:
# hacemos el calculo de matriz de covarianza por industria con lo que tenemos por mientras
# para los datos iniciales
Empresa = namedtuple('Empresa', 'nombre industria sector marketcap')
originales_df =  pd.read_csv("companies.csv", sep=",")
originales_df["ticker"] = originales_df["ticker"].str.lower()
originales_df = originales_df[['ticker', 'company name', 'industry','sector', 'market cap']]
lista_tickers_original = originales_df['ticker'].tolist()
lista_sectores_original = originales_df['sector'].tolist()


l_sector= []
l_rend=[]
# print(df)
# print(df["Sector"])
#dict que guardara las matrices de covarianza por sector

dict_matrices = dict()
for sector in set(lista_sectores_original):
    df_rendimientos = pd.DataFrame()  # data frame vacio que iremos rellenando
    for accion, df in dict_frames_nuevo.items():
        if sector != df['Sector'].iloc[0]:
            continue
        else:
            df_rendimientos[accion] = df['Rendimiento']

    df_mean = df_rendimientos.mean(axis=1)
    list_df = df_mean.tolist()
    l_rend.append(list_df)
    l_sector.append(sector)

dic_r_diario_x_industria= {}
cont= 0
# for i in l_sector:
#     dic_r_diario_x_industria[i]:l_rend[cont]
#     cont+=1
# print(dic_r_diario_x_industria)
# print(l_rend)
dff = pd.DataFrame()
for i in l_sector:
    print(cont)
    print(l_rend[cont])
    dff[i]=l_rend[cont]
    cont+=1
# print(dff)
dff.dropna(axis='columns')

0
[]
1
[0.0235376220515158, 0.006960024262242975, 0.010894461776732395, 0.012142220026676887, -0.0017992419548390915, -0.030465472412316105, -0.015455169055852655, 0.04087079385162628, -0.005902451942667774, -0.0016178614166592793, -0.011048219933919677, -0.0006176442340625239, 0.0009214162783699767, 0.0026088914292240304, -0.006842322672299844, -0.03189426620766824, 0.05045918738438795, -0.013178123009760411, 0.04252893763699121, 0.01323550305629258, -0.0033297099359491365, 0.011074972002509361, 0.008956078188669931, 0.023305100392359182, -0.003433889324229494, -0.0024080960884279687, -0.00758332662969707, 0.020527706188094526, -0.0047504819794039746, 0.004636175376850674, 0.011401613537777383, -0.0018643222692864424, -0.008020197286454108, 0.007964742928014773, 0.0019041085272876205, 0.00936668363920122, 0.030352320521729616, 0.018602100953875943, -0.006653621367132266, 0.0059185311077264375, 0.027248813655712488, 0.0032720455774870394, -0.011549969637010441, -0.0015675142752522005, 

,Industrials,Real Estate,Consumer Defensive,Energy,Basic Materials,Communication Services,Utilities,Consumer Cyclical,Financial Services,Technology,Healthcare
0,0.023538,-0.010540,-0.004054,-0.010328,-0.005627,-0.006509,-0.007990,-0.011209,-0.006847,-0.003989,-0.002251
1,0.006960,-0.003441,-0.008461,-0.029358,-0.001298,-0.011270,-0.010067,-0.007020,-0.008187,-0.001047,-0.012049
2,0.010894,0.000697,0.006464,-0.003231,-0.001754,-0.011359,0.005624,-0.014101,0.000327,-0.008185,0.005772
3,0.012142,0.015402,0.009295,-0.041198,0.005095,0.003244,0.006424,0.004506,0.003340,0.006840,0.003414
4,-0.001799,-0.004987,-0.003740,-0.012788,-0.002473,0.005367,-0.005367,0.007700,-0.005433,0.005897,-0.001966
...,...,...,...,...,...,...,...,...,...,...,...
77,0.002568,0.003603,-0.006078,0.014798,-0.001780,0.000261,-0.004359,0.000673,-0.004440,-0.009502,-0.025932
78,0.002238,0.011079,0.003336,0.029791,0.007255,-0.004308,0.003510,-0.007757,0.004529,0.002459,-0.007556
79,-0.010249,0.005503,-0.009210,0.001696,0.000846,0.008282,0.000213,-0.001168,0.001305,-0.002022,-0.008696
80,-0.006315,-0.000369,-0.012070,-0.007538,-0.004200,-0.007158,-0.009535,-0.007527,-0.005973,-0.010824,-0.022093
